# 설명 가능한 CNN
---
CNN을 구성하는 각 중간 계층부터 최종 분류까지 입력된 이미지에서 특성이 어떻게 추출되고 학습하는지 시각적으로 설명할 수 있어야 결과에 대한 신뢰성을을 얻을 수 있습니다.

총 두 가지 방법이 있는데 필터에 대한 시각화와 특성 맵에 대한 시각화가 있습니다.


### 1. 특성 맵 시각화
---

특성 맵은 입력 이미지 또는 다른 특성 맵처럼 필터를 입력에 적용한 결과입니다. 

따라서 특정 입력 이미지에 대한 특성 맵을 시각화한다는 의미는 특성 맵에서 입력 특성을 감지하는 방법을 이해할 수 있도록 돕는 것입니다.

먼저 이를 위해 가상 환경에 PIL을 추가해 줍시다

```pip install pillow```

특성 맵은 합성곱층을 입력 이미지와 필터를 연산하여 얻은 결과인데, 합성곱층에서 입력과 출력을 알 수 있다면 특성 맵을 추출할 수 있다는 의미입니다.


In [ ]:
class LayerActivations: # 반드시 합성곱 층에서만 사용해야 함, 아니 에초에 특성 맵이 뭐였는지 생각해보자
    features = []
    def __init__(self, model, layer_num): # 모델과 레이어의 넘버를 인계받는다 레이어의 넘버는 print(model)할때 볼 수 있다. 
        self.hook = model[layer_num].register_forward_hook(self.hook_fn)
    
    def hook_fn(self, module, input, output):
        self.features = output.detach().numpy()

    def remove(self):
        self.hook.remove()

1. ```self.hook = model[layer_num].register_forward_hook(self.hook_fn)``` 에 대하여

> hook에 대한 글 : https://daebaq27.tistory.com/65

> 파이토치 공식 글 : https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=hook#torch.nn.Module.register_forward_pre_hook



HOOk 기능은 쉽게 말해서 패키지 중간에 자기가 원하는 코드 끼워넣을 수 있는 부분 정도로 이해하면 될 듯하다! (register hook)

register_forward_hook: forward 호출 후에 forward output 계산 후 걸어두는 hook

``` py
# register_forward_hook should have the following signature 
hook(module, input, output) -> None or modified output
```

input은 positional arguments만 담을 수 있으며 (index 같은?) keyword arguments 등은 담을 수 없고, forward 서만 적용이 된다.
hook은 forward output 수정 가능, input 또한 수정 가능하지만 forward에는 영향 없음.

1. 용어 풀이집
    - 위치 인자 (positional argument): 키워드 인자가 아닌 인자.

    - 키워드 인자 (keyword argument): 함수 호출 때 식별자가 앞에 붙은 인자 (예를 들어, name=) 또는 ** 를 앞에 붙인 딕셔너리로 전달되는 인자.   
    예를 들어, 다음과 같은 complex() 호출에서 3 과 5 는 모두 키워드 인자다

    ```py
    complex(real=3, imag=5)
    complex(**{'real': 3, 'imag': 5})
    ```

</br> </br>

2. 사용 시기
    특성 맵을 시각화해서 CNN의 내부 구조를 볼 수 있다. 이미지가 신경망을 지나면서 신경망이 이미지를 어떻게 학습하는지 이미지로 볼 수 있게 하는 것

    이미지를 1개만 넣어서 CNN의 내부 구조를 봐서 필터와 특성 맵을 시각화한다. 제대로 학습하는지 CNN의 결과의 신뢰성을 직접 확인할 수 있다.

    이미지 1개를 흘러보내되 CNN의 학습에 반영하지 않도록 고정해야 한다. 
